In [22]:
import boto3
import json
from botocore.client import ClientError

In [3]:
sns = boto3.client("sns")

### Topics: listing, creating & deleting

In [4]:
# Create topic
response = sns.create_topic(Name="new_topic")
topic_arn = response["TopicArn"]

# List topics
response = sns.list_topics()
topics = response["Topics"]

# Delete topics
# sns.delete_topic(TopicArn=topic_arn)

### Subscriptions: listing, creating & deleting

In [8]:
# Create SMS subscription
response = sns.subscribe(TopicArn=topic_arn, Protocol="SMS", Endpoint="+59891324637")
subscription_arn = response["SubscriptionArn"]

# Create email subscription
response = sns.subscribe(TopicArn=topic_arn, Protocol="email", Endpoint="estebanpbuday@yahoo.es")
subscription_arn = response["SubscriptionArn"]

In [12]:
# List all subscriptions
response = sns.list_subscriptions()
subscriptions = response["Subscriptions"]
subscriptions

[{'SubscriptionArn': 'arn:aws:sns:us-east-1:763864518324:new_topic:9521d287-db8e-4a67-a847-20258ee7a234',
  'Owner': '763864518324',
  'Protocol': 'sms',
  'Endpoint': '+59891324637',
  'TopicArn': 'arn:aws:sns:us-east-1:763864518324:new_topic'},
 {'SubscriptionArn': 'PendingConfirmation',
  'Owner': '763864518324',
  'Protocol': 'email',
  'Endpoint': 'estebanpbuday@yahoo.es',
  'TopicArn': 'arn:aws:sns:us-east-1:763864518324:new_topic'}]

In [13]:
# List subscriptions by topic
response = sns.list_subscriptions_by_topic(TopicArn=topic_arn)
subscriptions = response["Subscriptions"]
subscriptions

[{'SubscriptionArn': 'PendingConfirmation',
  'Owner': '763864518324',
  'Protocol': 'email',
  'Endpoint': 'estebanpbuday@yahoo.es',
  'TopicArn': 'arn:aws:sns:us-east-1:763864518324:new_topic'},
 {'SubscriptionArn': 'arn:aws:sns:us-east-1:763864518324:new_topic:9521d287-db8e-4a67-a847-20258ee7a234',
  'Owner': '763864518324',
  'Protocol': 'sms',
  'Endpoint': '+59891324637',
  'TopicArn': 'arn:aws:sns:us-east-1:763864518324:new_topic'}]

In [14]:
# Delete subscription
# sns.unsubscribe(SubscriptionArn=subscription_arn)

In [15]:
# Delete multiple subscriptions (here: all SMS ones)
# for sub in subscriptions:
# 	if sub["Protocol"] == "sms":
# 		sns.unsubscribe(sub["SubscriptionArn"])

In [16]:
# Publishing to topics
sns.publish(TopicArn=topic_arn, 
            Message="message text", 
            Subject="subject used in emails only")

{'MessageId': '80f537b0-6721-5ab1-96b1-6b2fec84371b',
 'ResponseMetadata': {'RequestId': 'ea68dd6e-ca29-5fa5-b9ec-81f1418c520f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ea68dd6e-ca29-5fa5-b9ec-81f1418c520f',
   'content-type': 'text/xml',
   'content-length': '294',
   'date': 'Thu, 30 Mar 2023 13:06:45 GMT'},
  'RetryAttempts': 0}}

In [17]:
# Send a single SMS (no topic, no subscription needed)
sns.publish(PhoneNumber="+59898941069", 
            Message="message text")

{'MessageId': '5afd11cd-9690-52a6-b197-4d521d6c4155',
 'ResponseMetadata': {'RequestId': '3bc7c6f2-d95b-534c-91ea-9a6d52a780bf',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3bc7c6f2-d95b-534c-91ea-9a6d52a780bf',
   'content-type': 'text/xml',
   'content-length': '294',
   'date': 'Thu, 30 Mar 2023 13:08:11 GMT'},
  'RetryAttempts': 0}}

In [31]:
class SnsWrapper:
    """Encapsulates Amazon SNS topic and subscription functions."""
    def __init__(self, sns_resource):
        """
        :param sns_resource: A Boto3 Amazon SNS resource.
        """
        self.sns_resource = sns_resource

    def publish_text_message(self, phone_number, message):
        """
        Publishes a text message directly to a phone number without need for a
        subscription.

        :param phone_number: The phone number that receives the message. This must be
                             in E.164 format. For example, a United States phone
                             number might be +12065550101.
        :param message: The message to send.
        :return: The ID of the message.
        """
        try:
            response = self.sns_resource.meta.client.publish(
                PhoneNumber=phone_number, Message=message)
            message_id = response['MessageId']
            print(f"Published message to {phone_number}")
        except ClientError:
            print("Couldn't publish message to %s.", phone_number)
            raise
        else:
            return message_id

In [32]:
resource_sns = boto3.resource('sns')
cm = SnsWrapper(resource_sns)

In [33]:
cm.publish_text_message("59898941069", "dsdss")

Published message to 59898941069


'29a87be2-329a-5e25-8478-277dbccfdfe5'